# Twitter API

In [2]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [3]:
query = "(from:ZemmourEric) until:2023-03-08 since:2022-01-01"
tweets = []
limit = 10

In [4]:
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.user.username, tweet.rawContent])
        
print(tweets[2][2])

Mettre l'économie à genoux, c'est détruire le travail, c'est détruire le pays. C'est leur objectif. Et ils n'en ont même pas honte.

https://t.co/7GrgoiZrdA


# Roberta Twitter Analysis (2022)

## Start

### Setting the admin mod

source : https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [5]:
import pyuac

def main():
    print("Do stuff here that requires being run as an admin.")
    # The window will disappear as soon as the program exits!
    input("Press enter to close the window. >")

if __name__ == "__main__":
    if not pyuac.isUserAdmin():
        print("Re-launching as admin!")
        pyuac.runAsAdmin()
    else:        
        main()

Re-launching as admin!


### Import

#### Model

In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

## Process

## Preprocess

In [7]:
# Preprocess text (username and link placeholders)

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [8]:
tweets_preprocessed = []
for tweet in tweets:
    preprocess_tweet = preprocess(tweet[2])
    tweets_preprocessed.append(preprocess_tweet)

print(pos_tweet := tweets[5][2]) # positif
print(neg_tweet := tweets[2][2]) # négatif

Bravo à eux. Le dialogue entre les jeunes de droite est la première étape avant l’union des droites.
@MagLincorrect https://t.co/Oj1ysohqBX
Mettre l'économie à genoux, c'est détruire le travail, c'est détruire le pays. C'est leur objectif. Et ils n'en ont même pas honte.

https://t.co/7GrgoiZrdA


### Initialize Model

In [9]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest" # path des jsons précédents
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

('cardiffnlp/twitter-roberta-base-sentiment-latest\\tokenizer_config.json',
 'cardiffnlp/twitter-roberta-base-sentiment-latest\\special_tokens_map.json',
 'cardiffnlp/twitter-roberta-base-sentiment-latest\\vocab.json',
 'cardiffnlp/twitter-roberta-base-sentiment-latest\\merges.txt',
 'cardiffnlp/twitter-roberta-base-sentiment-latest\\added_tokens.json',
 'cardiffnlp/twitter-roberta-base-sentiment-latest\\tokenizer.json')

In [10]:
text = "I love my wife !"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) positive 0.9838
2) neutral 0.0112
3) negative 0.0049


In [11]:
text = "I hate my wife !"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) negative 0.9249
2) neutral 0.0624
3) positive 0.0127


In [19]:
text = "We all lost"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


1) negative 0.736
2) neutral 0.2322
3) positive 0.0318
